In [5]:
import pandas as pd
import numpy as np
import requests
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
plt.style.use("seaborn")

/home/pnietoramos/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Defining Features and Classifier

In [6]:
df_all = pd.read_csv("Trees_clean.csv", index_col=False)

In [7]:
df_all.head(1)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type
0,2596,51,3,258,0,510,221,232,148,6279,5,29


In [8]:
X = df_all.drop("Cover_Type",axis=1)
y = df_all["Cover_Type"]

In [9]:
#Splitting the set (test = 0.3) in a stratified manner
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30,stratify=y, random_state=4444)

In [10]:
#scaling the features
sca = StandardScaler()
X_train_sca = sca.fit_transform(X_train)
X_test_sca = sca.transform(X_test);

In [11]:
RandomForestClassifier().get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [12]:
def grid_estimator(estimator, param_grid, X_train, y_train, X_test, y_test, cv=5):
    grid = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv, n_jobs=1)
    grid.fit(X_train, y_train)
    print("Accuracy training: " + str(grid.best_score_))
    y_pred = grid.predict(X_test)
    print("Accuracy test: " + str(accuracy_score(y_pred,y_test)))
    print(grid.best_estimator_)


In [13]:
def conf_trees(y, y_pred):
    trees_confusion = confusion_matrix(y, y_pred)
    plt.figure(dpi=180)
    sns.heatmap(trees_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
               xticklabels=["1","2","3","4","5","6","7"],
               yticklabels=["1","2","3","4","5","6","7"]);
    plt.xlabel('prediction')
    plt.ylabel('actual');

### Trying SVC with default values 

In [ ]:
param_grid = {}
grid_estimator(SVC(), param_grid, X_train_sca, y_train, X_test_sca, y_test)

In [ ]:
conf_trees(y_test, y_pred)

In [ ]:
print("Recall for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_test[y_test==(i+1)])))

In [ ]:
print("Precision for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_pred[y_pred==(i+1)])))

### Selecting features with importance >0.05  (Random Forest)

In [ ]:
col_in = ["Elevation","Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points", "Soil_Type",
          "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"]
X_train_in = X_train[col_in]
X_test_in = X_test[col_in]
sca = StandardScaler()
X_train_in_sca = sca.fit_transform(X_train_in)
X_test_in_sca = sca.transform(X_test_in);

In [ ]:
rf = SVC()
rf.fit(X_train_in_sca, y_train)
y_pred = rf.predict(X_test_in_sca)

In [ ]:
print("Recall for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_test[y_test==(i+1)])))

In [ ]:
print("Precision for cover:")
for i in range(7):
    print("Type " + str(i+1)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_pred[y_pred==(i+1)])))

### Hyperparameters tunning

In [ ]:
col_in = ["Elevation","Horizontal_Distance_To_Roadways","Horizontal_Distance_To_Fire_Points", "Soil_Type",
          "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"]
X_train_in = X_train[col_in]
X_test_in = X_test[col_in]
sca = StandardScaler()
X_train_in_sca = sca.fit_transform(X_train_in)
X_test_in_sca = sca.transform(X_test_in);

In [ ]:
param_grid = {"C": [0.1,1, 10, 100, 1000], "gamma": [1/15, 1/10, 1/5, 1, 10]}
grid_estimator(SVC(), param_grid, X_train_in_sca, y_train, X_test_in_sca, y_test)